In [ ]:
import pandas as pd
import time
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import joblib
%matplotlib inline

In [ ]:
service = pd.read_csv("/data/zkp/buy2/5.19/allservice.csv")
# buy2 = buy2[buy2.throughput_type=="HSF"]

In [ ]:
service.sample_time__m = service.sample_time__m.apply(lambda x:int(time.mktime(time.strptime(x, "%Y-%m-%d %H:%M:%S"))))
service.rename(columns={"sample_time__m": "sample_time"}, inplace=True)
service.drop(columns=["throughput_type"], inplace=True)

In [ ]:
# buy2[(buy2.qps__Q_s>=16) & (buy2.qps__Q_s<=17)]
service

In [ ]:
machine = pd.read_csv("/data/zkp/buy2/5.19/machine.csv")
print(machine.isnull().sum())

In [ ]:
# machine.head(1)

In [ ]:
# pod_meta = pd.read_csv("/data/zkp/buy2/5.19/pod_meta.csv")
pod_meta.columns

In [ ]:
service = pd.merge(service, pod_meta[["nc_sn", "container_ip", "container_app_group_name"]], on=["container_ip"], how="left")

In [ ]:
service = service[~pd.isna(service.rt__ms_Q)]

In [ ]:
service = service[service.sample_time<1652895000]

In [ ]:
service = service[~((service.sample_time>=1652891400) & (service.sample_time<=1652891800))]

In [ ]:
service

In [ ]:
test_nc_sn = {}
temp = pod_meta[pod_meta.container_app_group_name=="buy2host"]
container_ip =temp.container_ip.values
nc_sn = temp.nc_sn.values
for i in range(temp.shape[0]):
    test_nc_sn[container_ip[i]] = nc_sn[i]

In [ ]:
ay49c = pod_meta[pod_meta.container_app_name=="ay49c"]
test_batch_ip = {}
for key in test_nc_sn:
    for i in range(ay49c.shape[0]):
        if ay49c.nc_sn.values[i] == test_nc_sn[key]:
            test_batch_ip[key] = ay49c.container_ip.values[i]
            break

In [ ]:
test_nc_ip = {}
temp = pod_meta[pod_meta.container_app_group_name=="buy2host"]
container_ip =temp.container_ip.values
nc_ip = temp.nc_ip.values
for i in range(temp.shape[0]):
    test_nc_ip[container_ip[i]] = nc_ip[i]

In [ ]:
onoffline = pd.read_csv("/data/zkp/buy2/service/onoffline2.csv")
online = onoffline[onoffline.Task_Type=="Online"]
offline = onoffline[onoffline.Task_Type=="Offline"]

In [ ]:
offline = offline[["sample_time", "nc_sn", "cpu_util__pct"]]
offline.rename(columns={"cpu_util__pct": "offline_cpu_util__pct"}, inplace=True)

In [ ]:
def readCSVtype():
    file_path = "/data/zkp/buy2/5.12/DATA.csv"
    data = []
    with open(file_path, 'r') as f:
        print(f.readline())

        while True:
            line = f.readline()
            if not line:
                break
            temp = []
            s = line.split(',')
            ss = s[7][2:-1].split()
            for i in ss:
                temp.append(int(i))
            while len(temp) < 60:
                line = f.readline()
                index = line.find(']')
                if index != -1:
                    line = line[:index]
                ss = line.split()
                for i in ss:
                    temp.append(int(i))
            while True:    
                line = f.readline()
                if line.find(']') != -1:
                    break
            data.append(temp)

In [ ]:
data = pd.merge(data, offline, on=["sample_time", "nc_sn"], how="left")

In [ ]:
# DATA = pd.read_csv("/data/zkp/buy2/5.12/DATA.csv")
# DATA_X = np.load("/data/zkp/buy2/5.12/DATA.npy")
DATA = pd.read_csv("/data/zkp/buy2/5.19/DATA.csv")
DATA_X = np.load("/data/zkp/buy2/5.19/DATA.npy")

In [ ]:
DATA = pd.merge(DATA, pod_meta[["nc_sn", "container_ip"]], on=["container_ip"], how="left")

In [ ]:
DATA

In [ ]:
offline = offline[~pd.isna(offline.cpu_util__pct)]

In [ ]:
temp = pd.merge(service, offline[["sample_time", "nc_sn", "cpu_util__pct"]], on=["sample_time", "nc_sn"], how="left")

In [ ]:
temp = temp[~pd.isna(temp.cpu_util__pct_y)]

In [ ]:
temp

In [ ]:
result = []

In [ ]:
for app in temp.container_app_group_name.unique():
    df = temp[temp.container_app_group_name==app]
    if df.container_ip.unique().shape[0] >= 10:
        X = []
        for i in df.cpu_util__pct_y.values:
            X.append(i)
        X = np.asarray(X).reshape(-1, 1)
        y = df.rt__ms_Q.values
        p = []
        for i in range(10, 101, 10):
            p.append(np.percentile(y, i))
        y_new = []
        for i in y:
            for j in range(1, 11):
                if i <= p[j-1]:
                    y_new.append(j)
                    break
        y = np.asarray(y_new)
        model = RandomForestClassifier(n_estimators=100)
        X_train, X_test, y_train, y_test = train_test_split(X, y)
        model.fit(X_train, y_train)
        predict = model.predict(X_test)
        error = abs(predict - y_test)
        print(app, df.shape[0], error[error<3].shape[0] / error.shape[0])
        result.append([df.container_ip.unique().shape[0], error[error<3].shape[0] / error.shape[0]])

In [ ]:
result = np.asarray(result)

In [ ]:
np.mean(result[:, 1]), result.shape

In [ ]:
np.savetxt("/data/zkp/figure/sl_total_baseline.txt", result)

In [ ]:
onoffline = pd.read_csv("/data/zkp/buy2/service/onoffline.csv")
onoffline = onoffline[onoffline.Task_Type=="Offline"]
onoffline = onoffline[~pd.isna(onoffline.CPU_time__s)]

In [ ]:
DATA.sample_time.min(), DATA.sample_time.max()

In [ ]:
data = pd.merge(data, pod_meta[["nc_sn", "container_ip"]], on=["container_ip"], how="left")
# temp = pd.merge(DATA[["sample_time", "nc_sn"]],
#          onoffline[["sample_time", "nc_sn", "CPU_time__s"]], how="left", on=["sample_time", "nc_sn"])

In [ ]:
plt.plot(sorted(DATA.online_cpu_util__pct.values/96), np.arange(DATA.shape[0]) / (DATA.shape[0] - 1))
plt.plot(sorted(DATA.offline_cpu_util__pct.values/96), np.arange(DATA.shape[0]) / (DATA.shape[0] - 1))
plt.plot(sorted(DATA.machine_cpu_util__pct.values), np.arange(DATA.shape[0]) / (DATA.shape[0] - 1))
DATA = DATA["online_cpu_util__pct"]

In [ ]:
data = pd.read_csv("/data/zkp/buy2/service/DATA.csv")
data_x = np.load("/data/zkp/buy2/service/DATA.npy")

In [ ]:
for ip in data.container_ip.unique():
    temp = data[(data.container_ip==ip) & (data.sample_time>1644422400+3600*3) & (data.sample_time<1644422400+3600*6)]
    plt.scatter(temp.offline_cpu_util__pct, temp.rt__ms_Q)
    plt.ylim(100, 400)
    plt.show()


In [ ]:
plt.figure(figsize=(20, 8))
for ip in buy2host.container_ip.unique():
    qps = buy2host[buy2host.container_ip==ip].qps__Q_s.values
    t = buy2host[buy2host.container_ip==ip].sample_time.values
    plt.plot(t, qps)

In [ ]:
buy2host = temp[temp.container_app_group_name=="malldetailskiphost"]
plt.figure(figsize=(20, 8))
for ip in buy2host.container_ip.unique():
    qps = buy2host[buy2host.container_ip==ip].qps__Q_s.values
    t = buy2host[buy2host.container_ip==ip].sample_time.values
    plt.plot(t, qps)

离线未压制时间点 （1652893200， 1652894880）

In [ ]:
# data = DATA[(DATA.qps__Q_s>=7) & (DATA.qps__Q_s<=20)]
# data_x = DATA_X[(DATA.qps__Q_s>=7) & (DATA.qps__Q_s<=20)]
# data = DATA[type60[:, 59]!=0]
# data = DATA[(DATA.sample_time>=1644422400) & (DATA.sample_time<1644508800)]
# data_x = DATA_X[(DATA.sample_time>=1644422400) & (DATA.sample_time<1644508800)]
# data["rue"] = data.cpu_util__pct.values / data.qps__Q_s.values
# data.shape
data = DATA[~pd.isna(DATA.rt__ms_Q) & (DATA.qps__Q_s>7) & (DATA.sample_time>=1652889600+1200)]
data_x = DATA_X[~pd.isna(DATA.rt__ms_Q) & (DATA.qps__Q_s>7) & (DATA.sample_time>=1652889600+1200)]

In [ ]:
# _ = plt.hist(data.rue.values, bins=30, range=(0.02, 0.15))
# _ = plt.hist(data.machine_cpu_util__pct.values, bins=30, range=(0, 100))
_ = plt.hist(data.offline.values, bins=30, range=(2000, 2100))
# plt.show()
plt.figure(figsize=(20, 8))
# plt.plot(sorted(data.rt__ms_Q.values), np.arange(data.shape[0]) / (data.shape[0] - 1))
# plt.plot([0, 1500], [0.50, 0.50])
# plt.plot([0, 1500], [0.95, 0.95])
# plt.plot([0, 1500], [0.05, 0.05])
# plt.plot([0, 1500], [0.80, 0.80])
# plt.show()
# plt.scatter(data.qps__Q_s.values, data.rt__ms_Q.values)
# plt.xlim(15, 20)
# plt.ylim(1, 10)
# v = []
# for i in data.container_ip.unique():
    
#     temp = data[data.container_ip==i]
#     mean = np.mean(temp.rt__ms_Q.values)
#     std = np.std(temp.rt__ms_Q.values)
#     v.append(std/mean)
# plt.xlim(1, 2)
#     plt.plot(temp.sample_time, temp.rt__ms_Q)
# #     plt.xlabel("time")
# #     plt.ylabel("QPS")
#     print(temp[(temp.qps__Q_s<5)&(temp.sample_time>1652893200)])
#     break


In [ ]:
X_ = X[(data.offline>=1800) & (data.offline<=1900)]
y_ = y[(data.offline>=1800) & (data.offline<=1900)]
off = data[(data.offline>=1800) & (data.offline<=1900)].offline.values
for j in [59]:
    tempX = []
    tempY = []
    tempoff = []
    for i in range(6):
        if y_[X_[:, j]==i].shape[0] > 10:
#             print(y_[X_[:, j]==i].shape[0])
            tempX.append(i)
            tempY.append(np.mean(y_[X_[:, j]==i]))
            tempoff.append(np.mean(off[X_[:, j]==i]))
    print(j)
    print(tempY, tempoff)
    plt.plot(tempX, tempY)
    plt.show()

In [ ]:
pod_meta = pd.read_csv("/data/zkp/buy2/5.19/pod_meta.csv")

In [ ]:
data = pd.merge(data, pod_meta[["nc_sn", "container_ip"]], how="left", on=["container_ip"])

In [ ]:
data = pd.merge(data, offline[["sample_time", "nc_sn", "cpu_util__pct"]], how="left", on=["sample_time", "nc_sn"])

In [ ]:
data = data.dropna()

In [ ]:
data

In [ ]:
offline

In [ ]:
machine = pd.read_csv("/data/zkp/buy2/5.19/machine.csv")

In [ ]:
onoffline = pd.read_csv("/data/zkp/buy2/5.19/onoffline.csv")
online = onoffline[onoffline.Task_Type=="Online"]
offline = onoffline[onoffline.Task_Type=="Offline"]

In [ ]:
online

In [ ]:
data_x = data_x[(data.sample_time>=1644422400) & (data.sample_time<1644508800)]
data = data[(data.sample_time>=1644422400) & (data.sample_time<1644508800)]

In [ ]:
# metric = "rt__ms_Q"
# p75 = np.percentile(data[metric].values, 90)
# p25 = np.percentile(data[metric].values, 5)
# da = data[(data[metric] >= p75) | (data[metric] <= p25)]
# X = data_x[(data[metric] >= p75) | (data[metric] <= p25)]
# da = data
# X = data_x
X = np.asarray((data.values)[:, [-1]], dtype=float)
y = data.rt__ms_Q.values
# da.shape, p75, p25

In [ ]:
for i in range(9):
    print(np.mean(y[(y<p[i+1]) & (y>p[i])]), end=', ')

In [ ]:
p = []
for i in range(10, 101, 10):
    p.append(np.percentile(y, i))
y_new = []
for i in y:
    for j in range(1, 11):
        if i <= p[j-1]:
            y_new.append(j)
            break

In [ ]:
# X = np.hstack((X, qps.reshape(-1, 1)))
# X.shape
y = np.asarray(y_new)
y.shape

In [ ]:
model = RandomForestClassifier(n_estimators=100)
X_train, X_test, y_train, y_test = train_test_split(X, y)

model.fit(X_train, y_train)

In [ ]:
model.score(X_test, y_test)

In [ ]:
predict = model.predict(X_test)

In [ ]:
error = abs(predict - y_test)
plt.plot(sorted(error), np.arange(predict.shape[0]) / (predict.shape[0] - 1))
plt.xlim(0, 5)
plt.xlabel("error", fontsize=30)
plt.ylabel("CDF", fontsize=30)
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.grid(":")
plt.show()

In [ ]:
error[error<3].shape[0] / error.shape[0]

In [ ]:
temp = np.load("/data/zkp/figure/simulation/if2_BED.npy")
np.bincount(model.predict(temp.reshape(-1, 60)))

In [ ]:
p

In [ ]:
temp = np.load("/data/zkp/figure/simulation/base2_BED.npy")
np.bincount(model.predict(temp.reshape(-1, 60)))

In [ ]:
np.sum(base2)

In [ ]:
if2 = [   0, 2288, 2912, 1456, 1560, 1300, 1560, 2548, 2548, 1560,  156]
base2 = [   0,  887, 1847, 1869, 1795, 1697, 1873, 2216, 2486, 2338,  679]
p = [152.4086278578676, 180.53065741328675, 190.18831099665684, 197.62023979935213, 204.47526701760574, 
     211.55480364314505, 219.71990758889862, 230.76328394504978, 250.93011366088624, 329.14399794371144, ]

In [ ]:
sum = 0
for i in range(10):
    sum += p[i] * if2[i+1]
sum/np.sum(if2)

In [ ]:
sum = 0
for i in range(10):
    sum += p[i] * base2[i+1]
sum/np.sum(base2)

In [ ]:
sum = 0
for i in range(10):
    sum += (i+1) * if2[i+1]
sum/np.sum(if2)

In [ ]:
sum = 0
for i in range(10):
    sum += (i+1) * base2[i+1]
sum/np.sum(base2)

In [ ]:
plt.figure(figsize=(20, 8))
plt.plot(range(0, predict.shape[0], 50), predict[::50])
plt.plot(range(0, y_test.shape[0], 50), y_test[::50])
plt.ylim(0, 10)

In [ ]:
X.shape

In [ ]:
for i in range(y.shape[0]):
    print(y[i], predict[i])

In [ ]:
label = (da[metric].values >= p75)
label.shape
# label = []
# for i in da[metric].values:
#     if i <= p25 or i >= p75:
#         label.append(1)
#     else:
#         label.append(0)
# label = np.asarray(label)

In [ ]:
x0 = X[label==0]
y0 = label[label==0]
x1 = X[label==1]
y1 = label[label==1]
x0 = x0[:int(X.shape[0]*0.1)]
y0 = y0[:int(X.shape[0]*0.1)]
X_ = np.vstack((x0, x1))
label_ = np.hstack((y0, y1))
X_.shape, label_.shape

In [ ]:
np.bincount(label_)

In [ ]:
model = RandomForestClassifier()
X_train, X_test, y_train, y_test = train_test_split(X, label)
model.fit(X_train, y_train)
y_predict = model.predict(X_test)
model.score(X_test, y_test)

In [ ]:
cnt = [[0, 0], [0, 0]]
for i in range(y_predict.shape[0]):
    cnt[int(y_test[i])][int(y_predict[i])] += 1
print(cnt)

In [ ]:
model.fit(X, label)
joblib.dump(model, "/data/zkp/buy2/5.12/rt4.model")

In [ ]:
model_id = 1
model_name = ["rue", "rt__ms_Q"]
model_threhold = [[4.934319997902265, 3.9014909478170434], [519.8403993886907, 167.38771434255023]]
X = data_x
label = []
# label = data[model_name[model_id]].values>=model_threhold[model_id][0]
for i in data[model_name[model_id]].values:
    if i >= model_threhold[model_id][0] or i <= model_threhold[model_id][1]:
        label.append(1)
#     elif i >= model_threhold[model_id][0]:
#         label.append(1)
    else:
        label.append(0)

In [ ]:
label

In [ ]:
np.bincount(y_predict)

In [ ]:
model = joblib.load("/data/zkp/buy2/5.12/rt4.model")
y_predict = model.predict(X)

In [ ]:
cnt = [[0, 0], [0, 0], [0, 0]]
for i in range(y_predict.shape[0]):
    cnt[int(label[i])][int(y_predict[i])] += 1
print(cnt)
score = [cnt[0][0] / sum(cnt[0]), cnt[1][1] / sum(cnt[1]), cnt[0][0] / (cnt[0][0] + cnt[1][0]), cnt[1][1] / (cnt[1][1] + cnt[0][1])]
print(score)
print(2*score[0]*score[2] / (score[0] + score[2]), 2*score[1]*score[3] / (score[1]+score[3]))

In [ ]:
plt.hist(data[(y_predict==1) & (np.asarray(label)==2)].rt__ms_Q)
plt.xlabel("RT")
plt.ylabel("Frequency")

In [ ]:
# model.feature_importances_
cnt = [0, 0, 0]
for i in range(y_predict.shape[0]):
    if y_test[i] == 1:
        cnt[y_predict[i]] += 1
cnt, cnt[0] / sum(cnt)

In [ ]:
lr = LinearRegression()
lr.fit(X, data[metric].values)
lr.score(X, data[metric.values)
predict = lr.predict(X)
error = abs(predict - data[metric].values) / data[metric].values
plt.plot(sorted(error), np.arange(predict.shape[0]) / (predict.shape[0] - 1))

In [ ]:
# plt.hist(data[metric].values, bins=30)
plt.hist(data[data[metric]>=p75].qps__Q_s, bins=30)
plt.show()
plt.hist(data[data[metric]<=p25].qps__Q_s, bins=30)
plt.show()
plt.scatter(da[label==0].machine_cpu_util__pct, da[label==0].qps__Q_s)
plt.scatter(da[label==1].machine_cpu_util__pct, da[label==1].qps__Q_s)
plt.xlabel("Machine CPU")
plt.ylabel("QPS")
plt.legend(["0", "1"])

In [ ]:
heat0, heat1 = [], []
for i in range(y_test.shape[0]):
    if y_test[i] == y_predict[i]:
        if y_test[i] == 0:
            heat0.append(X_test[i])
        else:
            heat1.append(X_test[i])
heat0 = np.asarray(heat0)
heat1 = np.asarray(heat1)

In [ ]:
import seaborn as sns
vmin = 0
vmax = 60
cmap = "viridis_r"
fig, axes = plt.subplots(1, 2, figsize=(18, 8))
sns.heatmap(heat1.T, cmap=cmap, yticklabels=[], xticklabels=[], vmin=vmin, vmax=vmax, cbar=False, ax=axes[0])
# axes[0].xlabel("P80 Machine", fontsize=15)
# axes[0].ylabel("Task Type", fontsize=15)
# plt.show()
# plt.figure(figsize=(12, 8))
sns.heatmap(heat0.T, cmap=cmap, yticklabels=[], xticklabels=[], vmin=vmin, vmax=vmax, ax=axes[1])
# axes[1].xlabel("P20 Machine", fontsize=15)
# axes[1].ylabel("Task Type", fontsize=15)
plt.show()

In [ ]:
for i in range(heat0.shape[1]):
    print(i)
    plt.plot(sorted(heat0[:, i]), np.arange(heat0.shape[0]) / (heat0.shape[0] - 1))
    plt.plot(sorted(heat1[:, i]), np.arange(heat1.shape[0]) / (heat1.shape[0] - 1))
    plt.show()

In [ ]:
plt.scatter(X[:, 39], X[:, 59], c=label)
plt.legend(["0"])

In [ ]:
for i in range(60):
    x = []
    y = []
    temp = np.bincount(X[:, i])
    for j in range(temp.shape[0]):
        if temp[j] >= 5:
            cnt = np.bincount(label[X[:, i]==j])
            x.append(j)
            y.append(cnt[0] / temp[j])
    print(i)
    plt.plot(x, y)
    plt.show()


In [ ]:
temp = X[:, 59] 
cnt = [0 for i in range(9)]
po = [0 for i in range(9)]
for i in range(temp.shape[0]):
    if temp[i] < 9:
        cnt[temp[i]] += 1
        if label[i]:
            po[temp[i]] += 1
cnt, po

In [ ]:
X59 = da[X[:, 59]==4]
Y59 = label[X[:, 59]==4]
plt.scatter(X59[Y59==0].machine_cpu_util__pct, X59[Y59==0].qps__Q_s)
plt.scatter(X59[Y59==1].machine_cpu_util__pct, X59[Y59==1].qps__Q_s)
plt.xlabel("Machine CPU")
plt.ylabel("QPS")
plt.legend(["0", "1"])

In [ ]:
batch = pd.read_csv("/data/zkp/buy2/instance.csv")
batch.shape

In [ ]:
batch
# batch.type.value_counts()

In [ ]:
batch["job_task_name"] = batch.fuxi_job_name + batch.fuxi_task_name
batch["total_time"] = batch.end_time - batch.start_time
G = batch[["job_task_name", "fuxi_avg_cpu", "fuxi_avg_mem", "total_time"]].groupby(["job_task_name"]).mean()

In [ ]:
G

In [ ]:
task_type = {}
type_cnt = [0 for i in range(60)]
cpu = G.fuxi_avg_cpu.values
mem = G.fuxi_avg_mem.values
t = G.total_time.values
key = G.index.values
for i in range(G.shape[0]):
    if np.isnan(cpu[i]) or np.isnan(mem[i]) or np.isnan(t[i]):
        continue
    index = 0
    if cpu[i] < 50:
        index = 0
    elif 50 <= cpu[i] < 150:
        index = 1
    else:
        index = 2

    if mem[i] < 512:
        index = (index*5) + 0
    elif 512 <= mem[i] < 1536:
        index = (index*5) + 1
    elif 1536 <= mem[i] < 3072:
        index = (index*5) + 2
    elif 3072 <= mem[i] < 6144:
        index = (index*5) + 3
    else:
        index = (index*5) + 4

    if t[i] < 20:
        index = (index*4) + 0
    elif 20 <= t[i] < 40:
        index = (index*4) + 1
    elif 40 <= t[i] < 100:
        index = (index*4) + 2
    else:
        index = (index*4) + 3
    task_type[key[i]] = index
    type_cnt[index] += 1

In [ ]:
DATA

In [ ]:
temp = []
for i in batch.job_task_name.values:
    if i in task_type:
        temp.append(int(task_type[i]))
    else:
        temp.append(np.nan)
batch["type"] = temp

In [ ]:
batch[(batch.start_time>=1644505260) | (batch.end_time>1644505260)].to_csv("/data/zkp/buy2/simu_instance.csv", index=False)

In [ ]:
batch = batch[~np.isnan(batch.type)]
batch = batch[~((batch.start_time==0) | (batch.end_time==0))]
batch = batch[~pd.isna(batch.worker_address)]
batch.type = np.asarray(batch.type, dtype="int")
batch.columns, DATA.columns

In [ ]:
offline_data = []
sample_time = DATA.sample_time.values
container_ip = DATA.container_ip.values
cur_ip = container_ip[0]
df = batch[batch.worker_address==test_batch_ip[cur_ip]]
cnt = 0
for i in range(DATA.shape[0]):
#     print(i, end=',')
#     start = time.time()
    if cur_ip != container_ip[i]:
        print(cnt, end=',')
        cnt += 1
        cur_ip = container_ip[i]
        df = batch[batch.worker_address==test_batch_ip[cur_ip]]  
    maxpd = df[df.start_time<sample_time[i]]
    maxpd = maxpd[maxpd.end_time>sample_time[i]-60]
#     print(time.time()-start)
    offline_data.append(np.bincount(maxpd.type.values, minlength=60))


In [ ]:
offline_data

In [ ]:
# DATA["type"] = offline_data
DATA.drop(columns=["type"]).to_csv("/data/zkp/buy2/service/DATA.csv", index=False)

In [ ]:
np.save("/data/zkp/buy2/service/DATA.npy", np.asarray(offline_data))

In [ ]:
task_type = {}
type_cnt = [0 for i in range(240)]
cpu = G.fuxi_avg_cpu.values
mem = G.fuxi_avg_mem.values
t = G.total_time.values
cpu_max = G.fuxi_max_cpu.values
mem_max = G.fuxi_max_mem.values
key = G.index.values
for i in range(G.shape[0]):
    if np.isnan(cpu_max[i]) or np.isnan(mem_max[i]):
        continue
    index = 0
    if cpu[i] < 50:
        idnex = 0
    elif 50 <= cpu[i] < 150:
        index = 1
    else:
        index = 2

    if mem[i] < 512:
        index = (index*5) + 0
    elif 512 <= mem[i] < 1536:
        index = (index*5) + 1
    elif 1536 <= mem[i] < 3072:
        index = (index*5) + 2
    elif 3072 <= mem[i] < 6144:
        index = (index*5) + 3
    else:
        index = (index*5) + 4

    if t[i] < 20:
        index = (index*4) + 0
    elif 20 <= t[i] < 40:
        index = (index*4) + 1
    elif 40 <= t[i] < 100:
        index = (index*4) + 2
    else:
        index = (index*4) + 3
        
    if cpu[i] == 0 or mem[i] == 0:
        index = (index*4) + 0
    elif mem_max[i]/mem[i] < 1.5 and cpu_max[i]/cpu[i] < 2:
        index = (index*4) + 0
    elif mem_max[i]/mem[i] < 1.5 and cpu_max[i]/cpu[i] >= 2:
        index = (index*4) + 1
    elif mem_max[i]/mem[i] >= 1.5 and cpu_max[i]/cpu[i] < 2:
        index = (index*4) + 2
    else:
        index = (index*4) + 3
    
    task_type[key[i]] = index
    type_cnt[index] += 1

In [ ]:
type60 = []
for i in DATA.type60.values:
    type60.append(i)
type60 = np.asarray(type60)

In [ ]:
for i in range(60):
    print(i, sum(type60[:, i]))

In [ ]:
type240 = []
for i in DATA.type240.values:
    type240.append(i)
type240 = np.asarray(type240)

In [ ]:
for i in range(240):
    print(sum(type240[:, i]), end=', ')